## Day 19



In [138]:
import re

def readInput19(infile):
    with open(infile) as f:
        o = f.read().strip().split("\n\n")
        workflows = {}
        for w in o[0].split("\n"):
            k = w.strip("}").split("{")
            workflows[k[0]] = tuple([ tuple(c.split(":")) for c in k[1].split(",") ])        
        ratings = [ tuple([int(x) for x in re.findall("\d+",r)]) for r in o[1].split() ]
        return workflows, ratings

### Part 1

In [139]:
def process_workflow(r,w):
    x,m,a,s = r
    for rule in w:
        if len(rule)==1:
            return rule[0]
        else:
            if eval(rule[0]):
                return rule[1]
        
def evaluate_rating(r,workflows):
    w = "in"
    while True:
        w = process_workflow(r,workflows[w])
        if w=="A":
            return True
        if w=="R":
            return False

def part1(infile):
    workflows, ratings = readInput19(infile)
    return sum( [ sum(r) for r in ratings if evaluate_rating(r,workflows) ] )

In [140]:
print("Test 1:",part1("examples/example19.txt"))
print("Part 1:",part1("AOC2023inputs/input19.txt"))

Test 1: 19114
Part 1: 434147


### Part 2

Counting possible success path in recursive way, starting from the full (0,4000) interval and reducing them according to the workflow rules.

Intervals are stored as the `tuple` of (beginning, end) in a dictionary with keys = `['x','m','a','s']`.

In [226]:
def count_ratings(workflows,intervals,goal="in"):
    # in case the outcome is R, no rating combinatoion is valid
    if goal=="R":
        return 0
    
    # in case the outcome is A, the number of combinations is the production of the intervals
    if goal=="A":
        prod = 1
        for x,(b,e) in intervals.items():
            prod *= (e-b)+1
        return prod
    
    # otherwise, process rules recursively changing the interval accordingly
    rules = workflows[goal]
    fallback = rules[-1][0] # last item of rules, contain a fallback direction without conditions

    count = 0
    
    processFallback = True
    
    for rule,target in rules[:-1]: # loop on all the rules in current workflow excluding fallback
        var = rule[0]
        op  = rule[1]
        val = int(rule[2:])
        
        # current interval for variable concerned by workflow rule
        beg, end = intervals[var]
        
        # split the current interval in Pass and Fail intervals according to the current rule
        if op == "<":
            Pass = (beg, val-1)
            Fail = (val, end)
        else:
            Pass = (val+1, end)
            Fail = (beg, val)
        
        # process Pass and Fail intervals to check whether one of them is empty
        if Pass[0] <= Pass[1]: # not empty, pass it recursively
            intervals_new = dict(intervals) # copy interval dictionary
            intervals_new[var] = Pass # update copy
            count += count_ratings(workflows,intervals_new,target)
        if Fail[0] <= Fail[1]: # not empty, update the interval dictionary and move to next rule in for loop
            intervals = dict(intervals) # copy interval dictionary 
            intervals[var] = Fail
        else: # if the Fail interval is empty, the Pass interval covered all possibilities, 
              # so no need to process the next rules, can break the for loop
            processFallback = False # if loop broken, no need to process fallback
            break
    
    if processFallback:
        # count toward fallback using interval dictionlay as modified by rule processing loop
        # what handling the Fail intervals
        count += count_ratings(workflows,intervals, fallback) 
    
    return count

In [227]:
def part2(infile):
    workflows, ratings = readInput19(infile)
    intervals = { "x": (1,4000), "m": (1,4000), "a": (1,4000), "s": (1,4000) }
    return count_ratings(workflows,intervals)

In [230]:
print("Test 2:",part2("examples/example19.txt"))
print("Part 2:",part2("AOC2023inputs/input19.txt"))

Test 2: 167409079868000
Part 2: 136146366355609
